In [19]:
import os
from os import walk
import warnings
import sys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import mlflow
import mlflow.sklearn

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression

## Loading and processing data

In [4]:
df = pd.read_csv('project1_output.csv')

#### Converting cols to their appropriate types again because we lost it on the csv export

In [5]:
df["product_category_name"] = df["product_category_name"].astype('category')
df["order_status"] = df["order_status"].astype('category')
df["review_score"] = df["review_score"].astype('category')
df["payment_type"] = df["payment_type"].astype('category')
df["customer_zip_code_prefix"] = df["customer_zip_code_prefix"].astype('category')
df["customer_city"] = df["customer_city"].astype('category')
df["customer_state"] = df["customer_state"].astype('category')
df["seller_zip_code_prefix"] = df["seller_zip_code_prefix"].astype('category')
df["seller_city"] = df["seller_city"].astype('category')
df["seller_state"] = df["seller_state"].astype('category')

df["product_name_lenght"] = df["product_name_lenght"].astype('int64')
df["product_description_lenght"] = df["product_description_lenght"].astype('int64')
df["product_photos_qty"] = df["product_photos_qty"].astype('int64')
df["payment_installments"] = df["payment_installments"].astype('int64')
df["payment_sequential"] = df["payment_sequential"].astype('int64')

In [6]:
df.dtypes

order_id                           object
order_item_id                       int64
product_id                         object
seller_id                          object
shipping_limit_date                object
price                             float64
freight_value                     float64
product_category_name            category
product_name_lenght                 int64
product_description_lenght          int64
product_photos_qty                  int64
product_weight_g                  float64
product_length_cm                 float64
product_height_cm                 float64
product_width_cm                  float64
customer_id                        object
order_status                     category
order_purchase_timestamp           object
order_approved_at                  object
order_delivered_carrier_date       object
order_delivered_customer_date      object
order_estimated_delivery_date      object
review_id                          object
review_score                     c

#### Droping unnecessary columns
We only want to work with numerical values.

In [7]:
df = df.select_dtypes(exclude=['object'])

In [8]:
df.columns

Index(['order_item_id', 'price', 'freight_value', 'product_category_name',
       'product_name_lenght', 'product_description_lenght',
       'product_photos_qty', 'product_weight_g', 'product_length_cm',
       'product_height_cm', 'product_width_cm', 'order_status', 'review_score',
       'payment_sequential', 'payment_type', 'payment_installments',
       'payment_value', 'customer_zip_code_prefix', 'customer_city',
       'customer_state', 'seller_zip_code_prefix', 'seller_city',
       'seller_state', 'payment_value_norm', 'volume'],
      dtype='object')

We'll also drop the `payment_value` column because our model would simply infer our target value from it by subtracting it from the `price` column.

In [9]:
df = df.drop(columns=['payment_value'])

### Feature engineering

Here we'll one-hot encode all of our categorical columns, and then drop the original ones

In [10]:
df = pd.get_dummies(df)
df = df.select_dtypes(exclude=['category'])

In [11]:
for col in df.columns:
    print(col)
print(len(df.columns))

order_item_id
price
freight_value
product_name_lenght
product_description_lenght
product_photos_qty
product_weight_g
product_length_cm
product_height_cm
product_width_cm
payment_sequential
payment_installments
payment_value_norm
volume
product_category_name_Outros
product_category_name_agro_industria_e_comercio
product_category_name_alimentos
product_category_name_alimentos_bebidas
product_category_name_artes
product_category_name_artes_e_artesanato
product_category_name_artigos_de_festas
product_category_name_artigos_de_natal
product_category_name_audio
product_category_name_automotivo
product_category_name_bebes
product_category_name_bebidas
product_category_name_beleza_saude
product_category_name_brinquedos
product_category_name_cama_mesa_banho
product_category_name_casa_conforto
product_category_name_casa_conforto_2
product_category_name_casa_construcao
product_category_name_cds_dvds_musicais
product_category_name_cine_foto
product_category_name_climatizacao
product_category_name_c

customer_zip_code_prefix_15102
customer_zip_code_prefix_15105
customer_zip_code_prefix_15108
customer_zip_code_prefix_15110
customer_zip_code_prefix_15115
customer_zip_code_prefix_15120
customer_zip_code_prefix_15130
customer_zip_code_prefix_15140
customer_zip_code_prefix_15150
customer_zip_code_prefix_15154
customer_zip_code_prefix_15155
customer_zip_code_prefix_15160
customer_zip_code_prefix_15170
customer_zip_code_prefix_15190
customer_zip_code_prefix_15200
customer_zip_code_prefix_15220
customer_zip_code_prefix_15230
customer_zip_code_prefix_15240
customer_zip_code_prefix_15260
customer_zip_code_prefix_15265
customer_zip_code_prefix_15270
customer_zip_code_prefix_15275
customer_zip_code_prefix_15285
customer_zip_code_prefix_15290
customer_zip_code_prefix_15300
customer_zip_code_prefix_15310
customer_zip_code_prefix_15315
customer_zip_code_prefix_15320
customer_zip_code_prefix_15330
customer_zip_code_prefix_15340
customer_zip_code_prefix_15350
customer_zip_code_prefix_15355
customer

customer_zip_code_prefix_65680
customer_zip_code_prefix_65690
customer_zip_code_prefix_65693
customer_zip_code_prefix_65695
customer_zip_code_prefix_65700
customer_zip_code_prefix_65705
customer_zip_code_prefix_65706
customer_zip_code_prefix_65707
customer_zip_code_prefix_65709
customer_zip_code_prefix_65715
customer_zip_code_prefix_65725
customer_zip_code_prefix_65735
customer_zip_code_prefix_65740
customer_zip_code_prefix_65750
customer_zip_code_prefix_65755
customer_zip_code_prefix_65760
customer_zip_code_prefix_65765
customer_zip_code_prefix_65775
customer_zip_code_prefix_65780
customer_zip_code_prefix_65790
customer_zip_code_prefix_65800
customer_zip_code_prefix_65810
customer_zip_code_prefix_65830
customer_zip_code_prefix_65860
customer_zip_code_prefix_65870
customer_zip_code_prefix_65885
customer_zip_code_prefix_65895
customer_zip_code_prefix_65900
customer_zip_code_prefix_65901
customer_zip_code_prefix_65903
customer_zip_code_prefix_65907
customer_zip_code_prefix_65910
customer

customer_city_moncoes
customer_city_mondai
customer_city_mongagua
customer_city_monnerat
customer_city_monsenhor paulo
customer_city_monsenhor tabosa
customer_city_montadas
customer_city_montalvania
customer_city_montanha
customer_city_montanhas
customer_city_montauri
customer_city_monte alegre
customer_city_monte alegre de minas
customer_city_monte alegre de sergipe
customer_city_monte alegre do sul
customer_city_monte alto
customer_city_monte alverne
customer_city_monte aprazivel
customer_city_monte azul
customer_city_monte azul paulista
customer_city_monte belo
customer_city_monte belo do sul
customer_city_monte bonito
customer_city_monte carlo
customer_city_monte carmelo
customer_city_monte castelo
customer_city_monte formoso
customer_city_monte gordo
customer_city_monte mor
customer_city_monte santo
customer_city_monte santo de minas
customer_city_monte siao
customer_city_monte verde
customer_city_monteiropolis
customer_city_montenegro
customer_city_montes claros
customer_city_mon

Even though we generated over 22000 columns this way, we believe that our model will be powerful enough to filter out any unecessary data.

## Picking column for prediction

We chose the `freight_value` column so we can perform a regression in order to try to find it's value based on all of the columns we have available.

In [12]:
TARGET_VALUE = 'freight_value'

In [13]:
target_col = df[TARGET_VALUE]

In [14]:
target_col

0         13.29
1         19.93
2         17.87
3         12.79
4         18.14
          ...  
118290    43.41
118291    36.53
118292    16.95
118293     8.72
118294    12.79
Name: freight_value, Length: 118295, dtype: float64

In [15]:
df = df.drop(columns=[TARGET_VALUE])

## Separating prediction and test data

We'll split our data in a 60/20/20 ratio.

In [16]:
train, val, test = np.split(df.sample(frac=1), [int(.6*len(df)), int(.8*len(df))])
train_labels, val_labels, test_labels = (
    np.split(
        target_col, 
        [int(.6*len(target_col)), int(.8*len(target_col))])
)

# Picking 4 ML algorithms

We'll use the following 4 algorithms:

1. Linear regression
2. Multilayer perceptron (a shallow one)
3. random forests
4. lightgbm/xgboost

### Metrics function

In [18]:
# Evaluate metrics
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

### Enabling MLFlow autologging

In [22]:
mlflow.sklearn.autolog()
mlflow.tensorflow.autolog()

## Linear regression
Let's start off with linear regression, which is the most simple algorithm in our selection, and will serve as a baseline for the following algorithms.

In [20]:
def linear_regression(features, labels):
    train, val, test = features
    train_labels, val_labels, test_labels = labels
    
    warnings.filterwarnings("ignore")
    np.random.seed(40)

    # Start an MLflow run; the "with" keyword ensures we'll close the run even if this cell crashes
    with mlflow.start_run():
        reg = LinearRegression()
        reg.fit(train, train_labels)

        predicted_qualities = reg.predict(val)

        (rmse, mae, r2) = eval_metrics(val_labels, predicted_qualities)

        # Print out ElasticNet model metrics
        print("Linear regression model")
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        # Log mlflow attributes for mlflow UI
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)
        mlflow.sklearn.log_model(reg, "model")
        modelpath = "./mlflow/freight_value/model-linear-reg"
        mlflow.sklearn.save_model(lr, modelpath)

In [ ]:
def mlp(features, labels, params: dict):
    # hyper-parameters to test
    hidden_units = params["hidden_units"]
    lr = params["lr"]
    epochs = params["epochs"]
    
    train, val, test = features
    train_labels, val_labels, test_labels = labels
    
    warnings.filterwarnings("ignore")
    np.random.seed(40)
    
    # Start an MLflow run; the "with" keyword ensures we'll close the run even if this cell crashes
    with mlflow.start_run():
        normalizer = preprocessing.Normalization(axis=-1)
        normalizer.adapt(np.array(x_train))
        
        mlp_model = tf.keras.Sequential([
            normalizer,
            layers.Dense(units=hidden_units)
            layers.Dense(units=hidden_units)
            layers.Dense(units=hidden_units)
            layers.Dense(units=1)
        ])

        mlp_model.summary()
        
        mlp_model.compile(
            optimizer=tf.optimizers.Adam(learning_rate=lr),
            loss='mean_absolute_error'
        )

        history = mlp_model.fit(
            x_train, y_train,
            validation_data=(x_test, y_test),
            epochs=epochs
        )

        (rmse, mae, r2) = eval_metrics(val_labels, predicted_qualities)

        # Print out ElasticNet model metrics
        print("Linear regression model")
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        # Log mlflow attributes for mlflow UI
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)
        mlflow.sklearn.log_model(reg, "model")
        modelpath = "./mlflow/freight_value/model-linear-reg"
        mlflow.sklearn.save_model(lr, modelpath)